In [ ]:
const padHeaderLines = (match) => {
	const padLine = (line, length) => {
		const padLength = length - line.length;
		if (padLength > 0) {
			const [padLeft, padRight] = [Math.ceil(padLength / 2), Math.ceil(padLength / 2)];
			const [lineLeft, lineRight] = [
				line.slice(0, Math.floor(line.length / 2)),
				line.slice(Math.floor(line.length / 2))
			];
			// Two types of padding: '█' and '░'. Count amount of each to get relative ratio.
			const fadePad = lineLeft.match(/░+/u)?.pop().length ?? 0;
			const fullFadeRatio = fadePad === 0 ? 1 : (lineLeft.match(/░+/)?.pop().length ?? 0) / fadePad;
			let numFullPadLeft = Math.round((fullFadeRatio * padLeft) / (1 + fullFadeRatio)),
							numFadePadLeft = 0,
							numFullPadRight = Math.round((fullFadeRatio * padRight) / (1 + fullFadeRatio)),
							numFadePadRight = 0;
			if (fadePad > 0) {
				numFadePadLeft = padLeft - numFullPadLeft;
				numFadePadRight = padRight - numFullPadRight;
			} else {
				numFullPadLeft = padLeft;
				numFullPadRight = padRight;
			}
			numFullPadRight += padLength - (numFullPadLeft + numFadePadLeft + numFullPadRight + numFadePadRight);
			return [
				lineLeft.replace(/▌█/u, `▌${"█".repeat(numFullPadLeft + 1)}`).replace(/░/u, "░".repeat(numFadePadLeft + 1)),
				lineRight.replace(/█▐/u, `${"█".repeat(numFullPadRight + 1)}▐`).replace(/░/u, "░".repeat(numFadePadRight + 1))
			].join("");
		}
		return line;
	};
	const lines = match.split(/\n/s);
	const returnLines = [];
	let [maxIndex, maxLen] = [0, 0];
	lines.forEach((line, i) => {
		if (line.length > maxLen) {
			maxIndex = i;
			maxLen = line.length;
		}
	});
	lines.forEach((line) => {
		if (line.length < maxLen) {
			returnLines.push(padLine(line, maxLen));
		} else {
			returnLines.push(line);
		}
	});
	return returnLines.join("\n");
};
const multiLineStripper = (match, startBreak, contents, endBreak) => {
	

};
const REGEXPPATTERNS = new Map([
	[/(\r\n)?\s*?(\/\*DEVCODE\*\/.*?\/\*!DEVCODE\*\/)\s*?(\r\n)/gsm, multiLineStripper], // Strip developer code
	[/\/\* \*{4}▌.*?▐\*{4} \*\//s, padHeaderLines], // Pad header lines to same length
	[/(\r?\n?)[ \t]*\/\*{1,2}(?!~)(?:.|\r?\n|\r)*?\*\/[ \t]*(\r?\n?)/g, "$1$2"], // Strip multi-line comments unless they beginning with '/*~' or '/**~'
	[/(\r?\n?)[ \t]*\/\/~.*?$/gm, "$1"], // Strip single-line comments unless they begin with '//~'
	[/[ \t]*\/\/[ \t]*eslint.*$/gm, ""], // Strip eslint enable/disable single-line comments
	[/[ \t]*\/\/[ \t]*@ts.*$/gm, ""], // Strip TypeScript expect-error comments
	[/[ \t]*\/\*[ \t]*eslint[^*]*\*\/[ \t]*/g, ""], // Strip eslint enable/disable mult-line comments
	[/[ \t]*\/\/ no default.*$/gm, ""], // Strip '// no default'
	[/[ \t]*\/\/ falls through.*$/gm, ""], // Strip '// falls through'
	[/[ \t]*~$/gm, ""], // Strip '~' from end-of-lines (used for automatic region folding)
	[/#reg.*? /gs, ""], // Convert region headers to standard headers
	[/(\r?\n?)[ \t]*\/\/ #endreg.*[ \t]*\r?\n?/g, "\r\n"], // Remove region footers
	[/(\r?\n[ \t]*(?=\r?\n)){2,}/g, "\r\n"], // Strip excess blank lines
	[/[ \t]*\r?\n$/g, ""], // Strip whitespace from end of files
	[/^[ \t]*\r?\n/g, ""] // Strip whitespace from start of files
]);
console.log(REGEXPPATTERNS);

In [1]:
// The HTML representation of an instance of GameObject, defined in game API
class DOMObject {
	private readonly gameObj: GameObject; // The linked GameObject, with the render method
	protected onRender: { // stores data to be applied when the DOMObject is rendered
		set?: gsap.CSSProperties // style properties that can't be applied until object is rendered
		animateTo?: gsap.TweenVars // properties of animation to be fired on render
	}

	// ... etc ...

	public async renderObject(): Promise<void> {
		this.renderPromise = this.gameObj.renderToDOM(); // GameObject's async render method
		await this.renderPromise; // wait for the render to complete, then apply onRender data
		if (this.onRender.animateTo) {
			gsap.to(this.element, this.onRender.animateTo);
		}
		return this.renderPromise;
	}

	public animateTo(vars: gsap.TweenVars): gsap.core.Tween | false {
		if (this.gameObj.isRendered) {
			return gsap.to(this.element, vars) // returns a Tween, but only if it's rendered
		}
		this.onRender.animateTo = { // otherwise, updates onRender so the animation can
			...this.onRender.animateTo ?? {}, //      be fired when it is rendered ...
			...vars
		}
		return false; // ... but I can't return a Tween in this case.
	}
}

SyntaxError: Unexpected identifier